In [1]:
import requests
import time
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
def search_athletes_by_name(athlete_name):
    
    # Define the base URL and the API key
    url = "https://graphql-prod-4765.prod.aws.worldathletics.org/graphql"
    headers = {"x-api-key": "da2-ois4zbc7s5htnlsy2it7bfkr2q"}
    payload = {
        "operationName": "SearchCompetitors",
        "variables": {
            "query": athlete_name
        },
        "query": """
        query SearchCompetitors($query: String, $gender: GenderType, $disciplineCode: String, $environment: String, $countryCode: String) {
          searchCompetitors(query: $query, gender: $gender, disciplineCode: $disciplineCode, environment: $environment, countryCode: $countryCode) {
            aaAthleteId
            familyName
            givenName
            birthDate
            disciplines
            iaafId
            gender
            country
            urlSlug
            __typename
          }
        }
        """
    }


    response = requests.post(url, json=payload, headers=headers)
    
    if response.status_code == 200:
        try:
            json_data = response.json()
            athletes_data = json_data["data"]["searchCompetitors"]
            
            if athletes_data:
                first_athlete = athletes_data[0]
                df = pd.json_normalize(first_athlete)
                return df
            else:
                return f"No results found for athlete: {athlete_name}"
        except JSONDecodeError as e:
            return f"Error decoding JSON response: {str(e)}\nResponse content: {response.text[:200]}"
    else:
        return f"Request failed with status code: {response.status_code}"

In [4]:
athlete_name = "vaudois"
first_athlete_df = search_athletes_by_name(athlete_name)

id_athlete = first_athlete_df['aaAthleteId'].iloc[0]
first_athlete_df

,aaAthleteId,familyName,givenName,birthDate,disciplines,iaafId,gender,country,urlSlug,__typename
0,14812063,VAUDOIS,Aurelien,06 MAR 1998,"4x800 Metres Relay, 800 Metres, 800 Metres Sho...",404944,Men,FRA,france/aurelien-vaudois-014812063,AthleteSearchResult


In [33]:
first_athlete_df

,aaAthleteId,familyName,givenName,birthDate,disciplines,iaafId,gender,country,urlSlug,__typename
0,14209691,RUDISHA,David,17 DEC 1988,"800 Metres, 400 Metres, 600 Metres",209036,Men,KEN,kenya/david-rudisha-014209691,AthleteSearchResult


In [36]:
id_athlete

'14209691'

In [8]:
url = "https://graphql-prod-4765.prod.aws.worldathletics.org/graphql"
headers = {
     "accept": "*/*",
    "content-type": "application/json",
    "x-amz-user-agent": "aws-amplify/3.0.2",
    "x-api-key": "da2-ois4zbc7s5htnlsy2it7bfkr2q"
}

df_list = []

for year in range(1960,2025):
    payload = {
        "operationName": "GetSingleCompetitorResultsDate",
        "variables": {
            "resultsByYear": year,
            "resultsByYearOrderBy": "date",
            "id": id_athlete  
        },
        "query": """
        query GetSingleCompetitorResultsDate($id: Int, $resultsByYearOrderBy: String, $resultsByYear: Int) {
          getSingleCompetitorResultsDate(id: $id, resultsByYear: $resultsByYear, resultsByYearOrderBy: $resultsByYearOrderBy) {
            parameters {
              resultsByYear
              resultsByYearOrderBy
              __typename
            }
            activeYears
            resultsByDate {
              date
              competition
              venue
              indoor
              disciplineCode
              disciplineNameUrlSlug
              typeNameUrlSlug
              discipline
              country
              category
              race
              place
              mark
              wind
              notLegal
              resultScore
              remark
              __typename
            }
            __typename
          }
        }
        """
    }

    response = requests.post(url, json=payload, headers=headers)
    try:
       data = response.json()["data"]["getSingleCompetitorResultsDate"]["resultsByDate"]
       print(f"Successfully fetched data for year {year}")
    except:
        print(f"Failed to fetch data for year {year}")
        continue
    df = pd.json_normalize(data)
    df_list.append(df)

ava = pd.concat(df_list, ignore_index=True)
ava.head()

Failed to fetch data for year 1960
Failed to fetch data for year 1961
Failed to fetch data for year 1962
Failed to fetch data for year 1963
Failed to fetch data for year 1964
Failed to fetch data for year 1965
Failed to fetch data for year 1966
Failed to fetch data for year 1967
Failed to fetch data for year 1968
Failed to fetch data for year 1969
Failed to fetch data for year 1970
Failed to fetch data for year 1971
Failed to fetch data for year 1972
Failed to fetch data for year 1973
Failed to fetch data for year 1974
Failed to fetch data for year 1975
Failed to fetch data for year 1976
Failed to fetch data for year 1977
Failed to fetch data for year 1978
Failed to fetch data for year 1979
Failed to fetch data for year 1980
Failed to fetch data for year 1981
Failed to fetch data for year 1982
Failed to fetch data for year 1983
Failed to fetch data for year 1984
Failed to fetch data for year 1985
Failed to fetch data for year 1986
Failed to fetch data for year 1987
Failed to fetch data

,date,competition,venue,indoor,disciplineCode,disciplineNameUrlSlug,typeNameUrlSlug,discipline,country,category,race,place,mark,wind,notLegal,resultScore,remark,__typename
0,29 APR 2017,Chenove,Chenove (FRA),None,None,None,None,800 Metres,FRA,F,F,2.,1:58.55,None,False,797,,resultsByDateResults
1,28 APR 2018,Chenove,Chenove (FRA),None,None,None,None,800 Metres,FRA,F,F,1.,1:57.79,None,False,816,,resultsByDateResults
2,20 MAY 2018,"Montceau-les-Mines French Club Championships, ...",Montceau-les-Mines (FRA),None,None,None,None,800 Metres,FRA,F,F1,2.,1:58.30,None,False,803,,resultsByDateResults
3,01 JUN 2018,"Montbéliard Meeting National, Montbéliard",Montbéliard (FRA),None,None,None,None,800 Metres,FRA,F,F4,6.,1:57.94,None,False,812,,resultsByDateResults
4,10 JUN 2018,Dijon,Dijon (FRA),None,None,None,None,800 Metres,FRA,F,F1,3.,1:54.96,None,False,889,,resultsByDateResults


In [6]:
import requests
import pandas as pd
from json.decoder import JSONDecodeError

def get_athlete_results_by_name(athlete_name, start_year=1990, end_year=2025):
    """
    Récupère tous les résultats de compétition d'un athlète en le recherchant par son nom.
    
    Args:
        athlete_name (str): Nom de l'athlète à rechercher
        start_year (int, optional): Année de début pour la recherche de résultats. Par défaut 1990.
        end_year (int, optional): Année de fin pour la recherche de résultats. Par défaut 2025.
        
    Returns:
        pd.DataFrame: DataFrame contenant tous les résultats de l'athlète
        ou un message d'erreur si l'athlète n'est pas trouvé
    """
    # Recherche de l'ID de l'athlète par son nom
    athlete_info = search_athletes_by_name(athlete_name)
    
    # Vérification si un DataFrame a été retourné
    if isinstance(athlete_info, pd.DataFrame):
        if not athlete_info.empty:
            athlete_id = athlete_info['aaAthleteId'].iloc[0]
            athlete_name_full = f"{athlete_info['givenName'].iloc[0]} {athlete_info['familyName'].iloc[0]}"
            print(f"Athlète trouvé: {athlete_name_full} (ID: {athlete_id})")
            
            # Récupération des résultats avec l'ID de l'athlète
            results_df = get_athlete_competition_results(athlete_id, start_year, end_year)
            
            # Ajout des informations de l'athlète au DataFrame de résultats
            if not results_df.empty:
                for col in athlete_info.columns:
                    results_df[col] = athlete_info[col].iloc[0]
                
                return results_df
            else:
                return pd.DataFrame({'info': ['warning'], 'message': [f"Aucun résultat trouvé pour {athlete_name_full}"]})
        else:
            return pd.DataFrame({'info': ['error'], 'message': ["DataFrame d'athlète vide"]})
    else:
        # Si un message d'erreur est retourné
        return pd.DataFrame({'info': ['error'], 'message': [str(athlete_info)]})

def search_athletes_by_name(athlete_name):
    """
    Recherche un athlète par son nom via l'API World Athletics.
    
    Args:
        athlete_name (str): Nom de l'athlète à rechercher
        
    Returns:
        pd.DataFrame: DataFrame contenant les informations de l'athlète
        ou un message d'erreur si la recherche échoue
    """
    # Define the base URL and the API key
    url = "https://graphql-prod-4765.prod.aws.worldathletics.org/graphql"
    headers = {"x-api-key": "da2-ois4zbc7s5htnlsy2it7bfkr2q"}
    payload = {
        "operationName": "SearchCompetitors",
        "variables": {
            "query": athlete_name
        },
        "query": """
        query SearchCompetitors($query: String, $gender: GenderType, $disciplineCode: String, $environment: String, $countryCode: String) {
          searchCompetitors(query: $query, gender: $gender, disciplineCode: $disciplineCode, environment: $environment, countryCode: $countryCode) {
            aaAthleteId
            familyName
            givenName
            birthDate
            disciplines
            iaafId
            gender
            country
            urlSlug
            __typename
          }
        }
        """
    }
    
    try:
        response = requests.post(url, json=payload, headers=headers)
        response.raise_for_status()  # Lève une exception pour les codes d'erreur HTTP
        
        json_data = response.json()
        athletes_data = json_data.get("data", {}).get("searchCompetitors", [])
        
        if athletes_data:
            first_athlete = athletes_data[0]
            df = pd.json_normalize(first_athlete)
            return df
        else:
            return f"Aucun résultat trouvé pour l'athlète: {athlete_name}"
            
    except requests.exceptions.RequestException as e:
        return f"Erreur de requête: {str(e)}"
    except JSONDecodeError as e:
        return f"Erreur de décodage JSON: {str(e)}\nContenu de la réponse: {response.text[:200]}"
    except Exception as e:
        return f"Erreur inattendue: {str(e)}"

def get_athlete_competition_results(athlete_id, start_year=1990, end_year=2025):
    """
    Récupère tous les résultats de compétition d'un athlète par son ID.
    
    Args:
        athlete_id (int): ID de l'athlète
        start_year (int, optional): Année de début pour la recherche de résultats. Par défaut 1990.
        end_year (int, optional): Année de fin pour la recherche de résultats. Par défaut 2025.
        
    Returns:
        pd.DataFrame: DataFrame contenant tous les résultats de l'athlète
    """
    url = "https://graphql-prod-4765.prod.aws.worldathletics.org/graphql"
    headers = {
        "accept": "*/*",
        "content-type": "application/json",
        "x-amz-user-agent": "aws-amplify/3.0.2",
        "x-api-key": "da2-ois4zbc7s5htnlsy2it7bfkr2q"
    }
    
    df_list = []
    all_active_years = set()
    
    for year in range(start_year, end_year + 1):
        payload = {
            "operationName": "GetSingleCompetitorResultsDate",
            "variables": {
                "resultsByYear": year,
                "resultsByYearOrderBy": "date",
                "id": int(athlete_id)  # Assurez-vous que l'ID est un entier
            },
            "query": """
            query GetSingleCompetitorResultsDate($id: Int, $resultsByYearOrderBy: String, $resultsByYear: Int) {
              getSingleCompetitorResultsDate(id: $id, resultsByYear: $resultsByYear, resultsByYearOrderBy: $resultsByYearOrderBy) {
                parameters {
                  resultsByYear
                  resultsByYearOrderBy
                  __typename
                }
                activeYears
                resultsByDate {
                  date
                  competition
                  venue
                  indoor
                  disciplineCode
                  disciplineNameUrlSlug
                  typeNameUrlSlug
                  discipline
                  country
                  category
                  race
                  place
                  mark
                  wind
                  notLegal
                  resultScore
                  remark
                  __typename
                }
                __typename
              }
            }
            """
        }
        
        try:
            response = requests.post(url, json=payload, headers=headers)
            response.raise_for_status()
            
            data = response.json()
            
            # Vérifier la présence des clés nécessaires
            if "data" not in data or data["data"] is None:
                print(f"✗ Aucune donnée renvoyée pour l'année {year}")
                continue
                
            competitor_data = data["data"]["getSingleCompetitorResultsDate"]
            
            # Vérifier si competitor_data est None
            if competitor_data is None:
                print(f"✗ Aucune donnée disponible pour l'année {year}")
                continue
            
            # Récupérer les années actives si disponibles
            if "activeYears" in competitor_data and competitor_data["activeYears"]:
                active_years = competitor_data["activeYears"]
                all_active_years.update(active_years)
            else:
                active_years = []
            
            # Récupérer les résultats si disponibles
            if "resultsByDate" in competitor_data and competitor_data["resultsByDate"]:
                results_data = competitor_data["resultsByDate"]
                print(f"✓ Données récupérées pour l'année {year} ({len(results_data)} résultats)")
                
                df = pd.json_normalize(results_data)
                df['year'] = year  # Ajouter l'année comme colonne
                
                # Ajouter les années actives comme information
                if active_years:
                    df['active_years'] = ','.join(map(str, active_years))
                
                df_list.append(df)
            else:
                print(f"✗ Aucun résultat pour l'année {year}")
                
            # Optimisation: si on a des années actives, on peut sauter aux années actives futures
            if active_years and year not in active_years:
                active_future_years = [y for y in active_years if y > year]
                if active_future_years:
                    next_active_year = min(active_future_years)
                    if next_active_year - year > 1:
                        print(f"→ Passage direct à l'année active suivante: {next_active_year}")
                        year = next_active_year - 1  # -1 car la boucle incrémente year
                
        except requests.exceptions.RequestException as e:
            print(f"✗ Erreur de requête pour l'année {year}: {str(e)}")
            continue
        except (KeyError, TypeError, JSONDecodeError) as e:
            print(f"✗ Erreur de traitement des données pour l'année {year}: {str(e)}")
            continue
            
    if df_list:
        final_df = pd.concat(df_list, ignore_index=True)
        final_df['athlete_id'] = athlete_id  # Ajouter l'ID de l'athlète comme colonne
        if all_active_years:
            final_df['all_active_years'] = ','.join(map(str, sorted(all_active_years)))
        return final_df
    else:
        print("Aucun résultat n'a été trouvé pour cet athlète.")
        return pd.DataFrame()  # Retourner un DataFrame vide si aucun résultat n'est trouvé


In [9]:

# Exemple d'utilisation
results = get_athlete_results_by_name("aurelien vaudois", start_year=1960, end_year=2025)
# Affichage des résultats
print(results.shape)
results.head()

Athlète trouvé: Aurelien VAUDOIS (ID: 14812063)
✗ Aucune donnée disponible pour l'année 1960
✗ Aucune donnée disponible pour l'année 1961
✗ Aucune donnée disponible pour l'année 1962
✗ Aucune donnée disponible pour l'année 1963
✗ Aucune donnée disponible pour l'année 1964
✗ Aucune donnée disponible pour l'année 1965
✗ Aucune donnée disponible pour l'année 1966
✗ Aucune donnée disponible pour l'année 1967
✗ Aucune donnée disponible pour l'année 1968
✗ Aucune donnée disponible pour l'année 1969
✗ Aucune donnée disponible pour l'année 1970
✗ Aucune donnée disponible pour l'année 1971
✗ Aucune donnée disponible pour l'année 1972
✗ Aucune donnée disponible pour l'année 1973
✗ Aucune donnée disponible pour l'année 1974
✗ Aucune donnée disponible pour l'année 1975
✗ Aucune donnée disponible pour l'année 1976
✗ Aucune donnée disponible pour l'année 1977
✗ Aucune donnée disponible pour l'année 1978
✗ Aucune donnée disponible pour l'année 1979
✗ Aucune donnée disponible pour l'année 1980
✗ Aucun

,date,competition,venue,indoor,disciplineCode,disciplineNameUrlSlug,typeNameUrlSlug,discipline,country,category,...,athlete_id,all_active_years,aaAthleteId,familyName,givenName,birthDate,disciplines,iaafId,gender,urlSlug
0,29 APR 2017,Chenove,Chenove (FRA),None,None,None,None,800 Metres,FRA,F,...,14812063,"2017,2018,2019,2020,2021,2022,2023,2024,2025",14812063,VAUDOIS,Aurelien,06 MAR 1998,"4x800 Metres Relay, 800 Metres, 800 Metres Sho...",404944,Men,france/aurelien-vaudois-014812063
1,28 APR 2018,Chenove,Chenove (FRA),None,None,None,None,800 Metres,FRA,F,...,14812063,"2017,2018,2019,2020,2021,2022,2023,2024,2025",14812063,VAUDOIS,Aurelien,06 MAR 1998,"4x800 Metres Relay, 800 Metres, 800 Metres Sho...",404944,Men,france/aurelien-vaudois-014812063
2,20 MAY 2018,"Montceau-les-Mines French Club Championships, ...",Montceau-les-Mines (FRA),None,None,None,None,800 Metres,FRA,F,...,14812063,"2017,2018,2019,2020,2021,2022,2023,2024,2025",14812063,VAUDOIS,Aurelien,06 MAR 1998,"4x800 Metres Relay, 800 Metres, 800 Metres Sho...",404944,Men,france/aurelien-vaudois-014812063
3,01 JUN 2018,"Montbéliard Meeting National, Montbéliard",Montbéliard (FRA),None,None,None,None,800 Metres,FRA,F,...,14812063,"2017,2018,2019,2020,2021,2022,2023,2024,2025",14812063,VAUDOIS,Aurelien,06 MAR 1998,"4x800 Metres Relay, 800 Metres, 800 Metres Sho...",404944,Men,france/aurelien-vaudois-014812063
4,10 JUN 2018,Dijon,Dijon (FRA),None,None,None,None,800 Metres,FRA,F,...,14812063,"2017,2018,2019,2020,2021,2022,2023,2024,2025",14812063,VAUDOIS,Aurelien,06 MAR 1998,"4x800 Metres Relay, 800 Metres, 800 Metres Sho...",404944,Men,france/aurelien-vaudois-014812063


In [ ]:
import requests
import pandas as pd
from json.decoder import JSONDecodeError
import concurrent.futures
import time
from tqdm.notebook import tqdm

def get_athlete_results_by_name(athlete_name, start_year=1960, end_year=2025, use_threading=True, max_workers=10):
    """
    Récupère tous les résultats de compétition d'un athlète en le recherchant par son nom.
    Version optimisée avec multithreading pour accélérer le scraping.
    
    Args:
        athlete_name (str): Nom de l'athlète à rechercher
        start_year (int, optional): Année de début pour la recherche de résultats. Par défaut 1960.
        end_year (int, optional): Année de fin pour la recherche de résultats. Par défaut 2025.
        use_threading (bool, optional): Utiliser le multithreading pour accélérer les requêtes. Par défaut True.
        max_workers (int, optional): Nombre maximum de workers pour le multithreading. Par défaut 10.
        
    Returns:
        pd.DataFrame: DataFrame contenant tous les résultats de l'athlète
        ou un message d'erreur si l'athlète n'est pas trouvé
    """
    # Mesurer le temps d'exécution
    start_time = time.time()
    
    # Recherche de l'ID de l'athlète par son nom
    athlete_info = search_athletes_by_name(athlete_name)
    
    # Vérification si un DataFrame a été retourné
    if isinstance(athlete_info, pd.DataFrame):
        if not athlete_info.empty:
            athlete_id = int(athlete_info['aaAthleteId'].iloc[0])
            athlete_name_full = f"{athlete_info['givenName'].iloc[0]} {athlete_info['familyName'].iloc[0]}"
            print(f"Athlète trouvé: {athlete_name_full} (ID: {athlete_id})")
            
            # Récupération des résultats avec l'ID de l'athlète
            results_df = get_athlete_competition_results(
                athlete_id, 
                start_year, 
                end_year, 
                use_threading=use_threading,
                max_workers=max_workers
            )
            
            # Mesurer le temps total d'exécution
            end_time = time.time()
            execution_time = end_time - start_time
            print(f"Temps d'exécution total: {execution_time:.2f} secondes")
            
            # Ajout des informations de l'athlète au DataFrame de résultats
            if not results_df.empty:
                for col in athlete_info.columns:
                    results_df[col] = athlete_info[col].iloc[0]
                
                return results_df
            else:
                return pd.DataFrame({'info': ['warning'], 'message': [f"Aucun résultat trouvé pour {athlete_name_full}"]})
        else:
            return pd.DataFrame({'info': ['error'], 'message': ["DataFrame d'athlète vide"]})
    else:
        # Si un message d'erreur est retourné
        return pd.DataFrame({'info': ['error'], 'message': [str(athlete_info)]})

def search_athletes_by_name(athlete_name):
    """
    Recherche un athlète par son nom via l'API World Athletics.
    
    Args:
        athlete_name (str): Nom de l'athlète à rechercher
        
    Returns:
        pd.DataFrame: DataFrame contenant les informations de l'athlète
        ou un message d'erreur si la recherche échoue
    """
    # Define the base URL and the API key
    url = "https://graphql-prod-4765.prod.aws.worldathletics.org/graphql"
    headers = {"x-api-key": "da2-ois4zbc7s5htnlsy2it7bfkr2q"}
    payload = {
        "operationName": "SearchCompetitors",
        "variables": {
            "query": athlete_name
        },
        "query": """
        query SearchCompetitors($query: String, $gender: GenderType, $disciplineCode: String, $environment: String, $countryCode: String) {
          searchCompetitors(query: $query, gender: $gender, disciplineCode: $disciplineCode, environment: $environment, countryCode: $countryCode) {
            aaAthleteId
            familyName
            givenName
            birthDate
            disciplines
            iaafId
            gender
            country
            urlSlug
            __typename
          }
        }
        """
    }
    
    try:
        response = requests.post(url, json=payload, headers=headers)
        response.raise_for_status()
        
        json_data = response.json()
        athletes_data = json_data.get("data", {}).get("searchCompetitors", [])
        
        if athletes_data:
            first_athlete = athletes_data[0]
            df = pd.json_normalize(first_athlete)
            return df
        else:
            return f"Aucun résultat trouvé pour l'athlète: {athlete_name}"
            
    except requests.exceptions.RequestException as e:
        return f"Erreur de requête: {str(e)}"
    except JSONDecodeError as e:
        return f"Erreur de décodage JSON: {str(e)}"
    except Exception as e:
        return f"Erreur inattendue: {str(e)}"

def fetch_year_data(athlete_id, year):
    """
    Fonction auxiliaire pour récupérer les résultats d'une année spécifique.
    Utilisée par le multithreading.
    
    Args:
        athlete_id (int): ID de l'athlète
        year (int): Année à récupérer
        
    Returns:
        tuple: (année, données de l'année, années actives)
    """
    url = "https://graphql-prod-4765.prod.aws.worldathletics.org/graphql"
    headers = {
        "accept": "*/*",
        "content-type": "application/json",
        "x-amz-user-agent": "aws-amplify/3.0.2",
        "x-api-key": "da2-ois4zbc7s5htnlsy2it7bfkr2q"
    }
    
    payload = {
        "operationName": "GetSingleCompetitorResultsDate",
        "variables": {
            "resultsByYear": year,
            "resultsByYearOrderBy": "date",
            "id": athlete_id  
        },
        "query": """
        query GetSingleCompetitorResultsDate($id: Int, $resultsByYearOrderBy: String, $resultsByYear: Int) {
          getSingleCompetitorResultsDate(id: $id, resultsByYear: $resultsByYear, resultsByYearOrderBy: $resultsByYearOrderBy) {
            parameters {
              resultsByYear
              resultsByYearOrderBy
              __typename
            }
            activeYears
            resultsByDate {
              date
              competition
              venue
              indoor
              disciplineCode
              disciplineNameUrlSlug
              typeNameUrlSlug
              discipline
              country
              category
              race
              place
              mark
              wind
              notLegal
              resultScore
              remark
              __typename
            }
            __typename
          }
        }
        """
    }
    
    try:
        response = requests.post(url, json=payload, headers=headers)
        response.raise_for_status()
        
        data = response.json()
        
        # Vérifier la présence des clés nécessaires
        if "data" not in data or data["data"] is None:
            return year, None, []
            
        competitor_data = data["data"]["getSingleCompetitorResultsDate"]
        
        # Vérifier si competitor_data est None
        if competitor_data is None:
            return year, None, []
        
        # Récupérer les années actives
        active_years = competitor_data.get("activeYears", [])
        
        # Récupérer les résultats
        results_data = competitor_data.get("resultsByDate", [])
        if not results_data:
            return year, None, active_years
            
        df = pd.json_normalize(results_data)
        df['year'] = year
        
        return year, df, active_years
        
    except Exception as e:
        return year, None, []

def get_athlete_competition_results(athlete_id, start_year=1990, end_year=2025, use_threading=True, max_workers=10):
    """
    Récupère tous les résultats de compétition d'un athlète par son ID.
    Version optimisée avec support du multithreading.
    
    Args:
        athlete_id (int): ID de l'athlète
        start_year (int, optional): Année de début pour la recherche de résultats. Par défaut 1990.
        end_year (int, optional): Année de fin pour la recherche de résultats. Par défaut 2025.
        use_threading (bool, optional): Utiliser le multithreading pour accélérer les requêtes
        max_workers (int, optional): Nombre maximum de workers pour le multithreading
        
    Returns:
        pd.DataFrame: DataFrame contenant tous les résultats de l'athlète
    """
    all_years = list(range(start_year, end_year + 1))
    df_list = []
    all_active_years = set()
    
    # ÉTAPE 1 : Récupérer seulement les années actives d'abord
    first_year = all_years[0]
    print(f"Recherche des années actives pour l'athlète ID: {athlete_id}...")
    _, _, active_years = fetch_year_data(athlete_id, first_year)
    
    # Si nous avons des années actives, ne récupérer que ces années
    if active_years:
        all_active_years.update(active_years)
        filtered_years = [y for y in all_years if y in active_years]
        print(f"Années actives trouvées: {sorted(active_years)}")
        print(f"Récupération des données pour {len(filtered_years)} années au lieu de {len(all_years)}")
    else:
        # Si pas d'années actives, continuer avec toutes les années
        filtered_years = all_years
        print("Aucune information sur les années actives. Récupération de toutes les années.")
    
    # ÉTAPE 2 : Récupérer les données pour les années filtrées
    if use_threading and len(filtered_years) > 1:
        print(f"Utilisation du multithreading avec {min(max_workers, len(filtered_years))} workers...")
        # Utiliser ThreadPoolExecutor pour le multithreading
        with concurrent.futures.ThreadPoolExecutor(max_workers=min(max_workers, len(filtered_years))) as executor:
            # Soumettre toutes les tâches et créer un dict pour suivre les résultats
            future_to_year = {
                executor.submit(fetch_year_data, athlete_id, year): year 
                for year in filtered_years
            }
            
            # Utiliser tqdm pour afficher une barre de progression
            for future in tqdm(concurrent.futures.as_completed(future_to_year), total=len(filtered_years), desc="Récupération des données"):
                year = future_to_year[future]
                try:
                    year, df, active_years = future.result()
                    if df is not None:
                        print(f"✓ Données récupérées pour l'année {year} ({len(df)} résultats)")
                        df_list.append(df)
                    
                    if active_years:
                        all_active_years.update(active_years)
                        
                except Exception as e:
                    print(f"✗ Erreur pour l'année {year}: {str(e)}")
    else:
        # Version séquentielle pour les cas simples
        for year in tqdm(filtered_years, desc="Récupération des données"):
            year, df, active_years = fetch_year_data(athlete_id, year)
            if df is not None:
                print(f"✓ Données récupérées pour l'année {year} ({len(df)} résultats)")
                df_list.append(df)
            
            if active_years:
                all_active_years.update(active_years)
                
    # ÉTAPE 3 : Consolidation des résultats    
    if df_list:
        final_df = pd.concat(df_list, ignore_index=True)
        final_df['athlete_id'] = athlete_id
        if all_active_years:
            final_df['all_active_years'] = ','.join(map(str, sorted(all_active_years)))
            
        # Ajouter des métriques sur les résultats
        print(f"\nRésumé:")
        print(f"• Nombre total de résultats: {len(final_df)}")
        print(f"• Années avec résultats: {sorted(final_df['year'].unique())}")
        print(f"• Épreuves: {sorted(final_df['discipline'].unique())}")
        
        return final_df
    else:
        print("Aucun résultat n'a été trouvé pour cet athlète.")
        return pd.DataFrame()

# Exemple d'utilisation
# results = get_athlete_results_by_name("Usain Bolt", use_threading=True, max_workers=15)
# results.head()

In [41]:

# Exemple d'utilisation
results = get_athlete_results_by_name("yared nuguse", use_threading=True, max_workers=15, start_year=1960, end_year=2025)
results.head()
print(results.shape)

Athlète trouvé: Yared NUGUSE (ID: 14798735)
Recherche des années actives pour l'athlète ID: 14798735...
Aucune information sur les années actives. Récupération de toutes les années.
Utilisation du multithreading avec 15 workers...


Récupération des données:   0%|          | 0/66 [00:00<?, ?it/s]

✓ Données récupérées pour l'année 2017 (1 résultats)
✓ Données récupérées pour l'année 2018 (14 résultats)
✓ Données récupérées pour l'année 2025 (7 résultats)
✓ Données récupérées pour l'année 2020 (5 résultats)
✓ Données récupérées pour l'année 2021 (12 résultats)
✓ Données récupérées pour l'année 2024 (18 résultats)
✓ Données récupérées pour l'année 2022 (15 résultats)
✓ Données récupérées pour l'année 2023 (17 résultats)
✓ Données récupérées pour l'année 2019 (16 résultats)

Résumé:
• Nombre total de résultats: 105
• Années avec résultats: [np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024), np.int64(2025)]
• Épreuves: ['1000 Metres', '1500 Metres', '1500 Metres Short Track', '3000 Metres', '3000 Metres Short Track', '5000 Metres', '5000 Metres Short Track', '800 Metres', 'Cross Country', 'Distance Medley', 'Distance Medley Short Track', 'Mile', 'Mile Road', 'Mile Short Track']
Temps d'exécution total: 2.92

In [42]:
pd.set_option('display.max_columns', None)  # Afficher toutes les colonnes
results.head(1)

,date,competition,venue,indoor,disciplineCode,disciplineNameUrlSlug,typeNameUrlSlug,discipline,country,category,race,place,mark,wind,notLegal,resultScore,remark,__typename,year,athlete_id,all_active_years,aaAthleteId,familyName,givenName,birthDate,disciplines,iaafId,gender,urlSlug
0,02 JUN 2017,"Somerville adidas Boost Boston Games, Somervil...","Somerville, MA (USA)",None,None,None,None,Mile,USA,B,F,6.,4:07.28,None,False,987,,AthleteSearchResult,2017,14798735,"2017,2018,2019,2020,2021,2022,2023,2024,2025",14798735,NUGUSE,Yared,01 JUN 1999,"One Mile Short Track, 1500 Metres, One Mile",359252,Men,united-states/yared-nuguse-14798735


In [18]:
from src.utils.http_utils import search_athletes

athletes = search_athletes('yohan durand')
selected = athletes[2]

seq = selected['seq']
name = selected['name']
club = selected['club']
sex = selected['sex']

In [22]:
import os
from sqlalchemy import create_engine

from src.utils.athlete_utils import get_all_athlete_results, save_results_to_postgres, save_athlete_info, clean_and_prepare_results_df
from dotenv import load_dotenv
load_dotenv()
db_url = os.getenv("DB_URL")

engine = create_engine(db_url)
df = get_all_athlete_results("48514356455449504356")

if not df.empty:
    df = clean_and_prepare_results_df(df, seq)
    save_athlete_info(seq, name, club, sex, engine)
    save_results_to_postgres(df, seq, engine)

Erreur lors de l'insertion batch : (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "results_seq_club_date_epreuve_tour_pl_perf_vt_niv_pts_ville_key"
DETAIL:  Key (seq, club, date, epreuve, tour, pl, perf, vt, niv, pts, ville, annee)=(48514356455449504356, 24034.0, 2025-04-13, Marathon, H.Stade, 17 (17M - 2M1), 2h14'44'' (2h14'44''), , N1, , Paris, 2025) already exists.

[SQL: INSERT INTO results (seq, club, date, epreuve, tour, pl, perf, vt, niv, pts, ville, annee) VALUES (%(seq_m0)s, %(club_m0)s, %(date_m0)s, %(epreuve_m0)s, %(tour_m0)s, %(pl_m0)s, %(perf_m0)s, %(vt_m0)s, %(niv_m0)s, %(pts_m0)s, %(ville_m0)s, %(annee_m0)s), (%(seq_m1)s, %(club_m1)s, %(date_m1)s, %(epreuve_m1)s, %(tour_m1)s, %(pl_m1)s, %(perf_m1)s, %(vt_m1)s, %(niv_m1)s, %(pts_m1)s, %(ville_m1)s, %(annee_m1)s), (%(seq_m2)s, %(club_m2)s, %(date_m2)s, %(epreuve_m2)s, %(tour_m2)s, %(pl_m2)s, %(perf_m2)s, %(vt_m2)s, %(niv_m2)s, %(pts_m2)s, %(ville_m2)s, %(annee_m2)s), (%(seq_m3)s, %(club_m3

In [24]:
df = get_all_athlete_results("48514356455449504356")
df.shape

(264, 12)

In [25]:
df = clean_and_prepare_results_df(df, seq)

In [12]:
# fast_scraper.py
import os, asyncio, io, csv
from datetime import datetime
from typing import List, Optional

import httpx, requests_cache, asyncpg, pandas as pd
from selectolax.parser import HTMLParser
from fastinference.core import patch  # tiny helper to add chainable methods

DB_URL = os.getenv("DB_URL")
HEADERS = {"User-Agent": "Mozilla/5.0  (fast-scraper/1.0)"}

################################################################################
# Networking helpers
################################################################################
# ❶ enable transparent on-disk cache (~400 µs look-ups for already-seen pages)
requests_cache.install_cache("athle_cache", expire_after=86_400)

async def fetch_text(client: httpx.AsyncClient, url: str) -> str:
    r = await client.get(url, follow_redirects=True, headers=HEADERS, timeout=30)
    r.raise_for_status()
    return r.text


################################################################################
# Scraping
################################################################################
def _parse_years(html: str) -> List[str]:
    tree = HTMLParser(html)
    return [
        opt.attributes["value"].split("saison=")[-1]
        for opt in tree.css("select.selectMain option")
        if "saison=" in opt.attributes.get("value", "")
    ]

async def get_athlete_years(seq: str) -> List[str]:
    url = f"https://bases.athle.fr/asp.net/athletes.aspx?base=bilans&seq={seq}"
    async with httpx.AsyncClient() as client:
        html = await fetch_text(client, url)
    return _parse_years(html)

def _extract_results_table(tables: List[pd.DataFrame]) -> Optional[pd.DataFrame]:
    # small guard in case the HTML layout changes
    return tables[3] if len(tables) > 3 else None

async def _fetch_year_results(
    client: httpx.AsyncClient, seq: str, year: str
) -> Optional[pd.DataFrame]:
    url = (
        "https://bases.athle.fr/asp.net/athletes.aspx"
        f"?base=resultats&seq={seq}&saison={year}"
    )
    html = await fetch_text(client, url)
    tables = pd.read_html(io.StringIO(html), header=0)
    df = _extract_results_table(tables)
    if df is not None:
        df["Annee"] = year
    return df

async def get_all_athlete_results(seq: str) -> pd.DataFrame:
    years = await get_athlete_years(seq)
    if not years:
        return pd.DataFrame()

    async with httpx.AsyncClient(http2=True, headers=HEADERS) as client:
        tasks = [_fetch_year_results(client, seq, y) for y in years]
        dfs = await asyncio.gather(*tasks)

    out = pd.concat([d for d in dfs if d is not None], ignore_index=True)
    out["seq"] = seq
    return out


################################################################################
# Data cleaning – fastinference patch gives us a neat chainable `.clean()` 💧
################################################################################
@patch
def clean_and_prepare(df: pd.DataFrame, seq: str) -> pd.DataFrame:
    col_map = {
        "Club": "club",
        "Date": "date",
        "Epreuve": "epreuve",
        "Tour": "tour",
        "Pl.": "pl",
        "Perf.": "perf",
        "Vt.": "vt",
        "Niv.": "niv",
        "Pts": "pts",
        "Ville": "ville",
        "Annee": "annee",
    }
    df = df.rename(columns=col_map)
    df["seq"] = seq

    # rebuild full date
    df["date"] = pd.to_datetime(
        df["date"].astype(str).str.zfill(5) + "/" + df["annee"].astype(str),
        format="%d/%m/%Y",
        errors="coerce",
    )

    # normalise object columns
    for c in ("club epreuve tour pl perf vt niv pts ville").split():
        if c in df:
            df[c] = df[c].astype(str).str.strip().fillna("")

    return df.where(pd.notnull(df), None)


################################################################################
# PostgreSQL persistence – COPY is ~10× faster than to_sql
################################################################################
async def save_results_to_postgres(df: pd.DataFrame, table: str = "results") -> int:
    if df.empty:
        return 0

    # keep column order identical to DB schema
    cols = [
        "seq",
        "club",
        "date",
        "epreuve",
        "tour",
        "pl",
        "perf",
        "vt",
        "niv",
        "pts",
        "ville",
        "annee",
    ]
    df = df.drop_duplicates(subset=cols)[cols]

    # COPY FROM - stream CSV straight from memory
    csv_buf = io.StringIO()
    df.to_csv(csv_buf, index=False, header=False, quoting=csv.QUOTE_MINIMAL)
    csv_buf.seek(0)

    conn = await asyncpg.connect(DB_URL)
    async with conn.transaction():
        await conn.copy_to_table(table, source=csv_buf, columns=cols)
    await conn.close()
    return len(df)


################################################################################
# Example entry-point
################################################################################
async def scrape_and_load(seq: str):
    all_df = (await get_all_athlete_results(seq)).clean_and_prepare(seq)
    n_inserted = await save_results_to_postgres(all_df)
    print(f"Inserted {n_inserted} fresh rows for athlete {seq}")


if __name__ == "__main__":
    import sys
    asyncio.run(scrape_and_load(sys.argv[1]))

ModuleNotFoundError: No module named 'fastinference.core'

In [1]:
from src.utils.file_utils import convert_time_to_seconds

print(convert_time_to_seconds("1'51''98"))

111.98
